In [1]:
from subprocess import Popen, PIPE, CalledProcessError
import re
import platform
import shutil
import os.path
import fileinput
import ipywidgets as widgets
from io import BytesIO
import xml.etree.cElementTree as ET
from functools import partial

options = {'stdout': PIPE, 'stderr': PIPE, 'bufsize' : 1, 'universal_newlines' : True, 'shell' : False}
if (platform.system() == 'Windows'):
    options['shell'] = True
    #Packages required to generate ssh keys in windows
    from cryptography.hazmat.primitives import serialization as crypto_serialization
    from cryptography.hazmat.primitives.asymmetric import rsa
    from cryptography.hazmat.backends import default_backend as crypto_default_backend

def callPopen(cmd):
    print(cmd)
    with Popen(cmd.split(),**options) as p:
        for line in p.stdout:
            print(line, end='')
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)

def isInstance(name):
    instanceExists=False
    ip=''
    with Popen('gcloud compute instances list'.split(),**options) as p:
        for line in p.stdout:
            if re.match('^{}'.format(name), line):
                instanceExists=True
                ip = line.strip().split()
                ip = ip[4]
        for line in p.stderr:
            print(line, end='')
        if p.returncode != (0 or None):
            raise CalledProcessError(p.returncode, p.args)
        return(instanceExists,ip)
    
            
def text_prepender(filename, text):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(text.rstrip('\r\n') + content)

def replace(file,pattern,replace):
    fileinput.close()
    for line in fileinput.input(file, inplace=True):
        print( re.sub(pattern,
                      replace,
                      line.rstrip()
                      ) 
             )
        
if (platform.system() == 'Windows'):
    def generateSSHKey(username,savePath):
        key = rsa.generate_private_key(
            backend=crypto_default_backend(),
            public_exponent=65537,
            key_size=2048
            )
        private_key = key.private_bytes(
            crypto_serialization.Encoding.PEM,
            crypto_serialization.PrivateFormat.TraditionalOpenSSL,
            crypto_serialization.NoEncryption()
            )
        public_key = key.public_key().public_bytes(
            crypto_serialization.Encoding.OpenSSH,
            crypto_serialization.PublicFormat.OpenSSH
            )
        public_file = os.path.join(savePath,username + '.pub')
        private_file = os.path.join(savePath,username)
        text_file = open(public_file, "w")
        text_file.write(public_key.decode('utf-8') + ' ' + username)
        text_file.close()
        text_file = open(private_file, "w")
        text_file.write(private_key.decode('utf-8'))
        text_file.close()
        print('Successfully created key pair')
            
if (platform.system() == 'Linux'):
    def generateSSHKey(username,savePath):
        p = Popen("echo 'yes' | ssh-keygen -t rsa -f {0}/{1} -C {1} -N '' ".format(savePath,username),
              stdout=PIPE,
              shell=True,
              stderr=PIPE
               )
        print(p.communicate())    

# Automated calibration for the GOTM/FABM model at Langtjern

## Pre-requisites

* gcloud command line utility
* google cloud account, you might need to be an admin for some of this to work, also this is not free
* ssh key setup in gitlab.au.dk
* fabric3 needs to be (pip) installed
* python modules listed in the first cell of this notebook

## Creating an instance on Google Cloud

In [2]:
instanceName = 'gotm-ssd'
username = 'tadhg'

createInstance = '''\
gcloud compute instances create {} \
--zone us-east1-b \
--image-family debian-9 \
--image-project debian-cloud \
--machine-type n1-standard-64 \
--boot-disk-size 375GB \
--boot-disk-type=pd-ssd
'''
#--boot-disk-size 200GB \
#--machine-type n1-highcpu-32  n1-standard-2\
deleteInstance = '''\
gcloud compute instances delete {} \
--zone us-east1-b \
'''

listInstances = '''gcloud compute instances list'''

addSSHKeys = '''gcloud compute instances add-metadata {} --zone us-east1-b --metadata-from-file ssh-keys={}'''

if (platform.system() == 'Linux'):
    keyDir = ('/home/jose-luis/.ssh/gotmKeys_feeagh')

if (platform.system() == 'Windows'):
    keyDir = ('c:/Users/jose_luis_guerrero/gotmKeys_feeagh')
    

ip=''
instanceExists,ip = isInstance(instanceName)

if instanceExists:
    print('Instance {} is {}'.format(instanceName,ip) )

isStarted = False
if instanceExists and ip == 'TERMINATED' :
    callPopen('gcloud compute instances start {} --zone us-east1-b'.format(instanceName))
    instanceExists,ip = isInstance(instanceName)
    isStarted = True
    print("Machine started and ip is {}".format(ip))

wasCreated=False
if not instanceExists and not isStarted:
    callPopen(createInstance.format(instanceName))
    wasCreated=True
    if os.path.exists(keyDir):
        shutil.rmtree(keyDir)
    os.mkdir(keyDir)
    generateSSHKey(username,keyDir)
    keyFile = os.path.join(keyDir,username + '.pub')
    text_prepender('{}/{}.pub'.format(keyDir,username), '{}:'.format(username) )
    callPopen(addSSHKeys.format(instanceName,keyDir + '/{}.pub'.format(username)))
    #callPopen('sed -i s/^{0}:// {1}/{0}.pub'.format(username,keyDir))
    replace(keyFile,"^{}:".format(username),"")
    ip=isInstance(instanceName)[1]
        #callPopen('chmod 400 {}'.format(keyDir))
        
print("The ip of {} is {}".format(instanceName,ip))

gcloud compute instances create gotm-ssd --zone us-east1-b --image-family debian-9 --image-project debian-cloud --machine-type n1-standard-64 --boot-disk-size 375GB --boot-disk-type=pd-ssd

NAME      ZONE        MACHINE_TYPE    PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
gotm-ssd  us-east1-b  n1-standard-64               10.142.0.2   35.237.12.54  RUNNING
Created [https://www.googleapis.com/compute/v1/projects/nivacatchment/zones/us-east1-b/instances/gotm-ssd].
(b"Generating public/private rsa key pair.\nYour identification has been saved in /home/jose-luis/.ssh/gotmKeys_feeagh/tadhg.\nYour public key has been saved in /home/jose-luis/.ssh/gotmKeys_feeagh/tadhg.pub.\nThe key fingerprint is:\nSHA256:Tr3juoVY6YkTQBt/96gyaH/ORCAnHaRMXf7rNRic0GI tadhg\nThe key's randomart image is:\n+---[RSA 2048]----+\n|    +oo..        |\n|   + *.o .       |\n|    B = E o      |\n|     = + O +     |\n|      . S B .    |\n|     . X + =     |\n|    o * B * o    |\n|   . . *.+ o .   |\n|      .o=oo      

## Updating fabfile.py with credentials and ip

In [3]:
if (platform.system() == "Linux"):
    callPopen("sed -i s/^env\.hosts.*/env.hosts=\['{}']/ fabfile.py".format(ip))
    callPopen("sed -i s/^env\.user.*/env.user=\'{}\'/ fabfile.py".format(username))
    callPopen("sed -i s$^env\.key_filename.*$env\.key_filename='{}'$ fabfile.py".format(keyDir + '/' + username))
    callPopen("sed -i s/^env\.roledefs.*/env.roledefs={{\\'{}\\':[\\'{}\\'],/ fabfile.py".format('stage',ip))

fabfile = os.path.join("C:\\Users\\jose_luis_guerrero\\Envs\\mylai\\prognos_calibration","fabfile.py")
    
if (platform.system() == "Windows"):    
    replace(fabfile, "^env\.hosts.*",         "env.hosts=['{}']".format(ip))
    replace(fabfile, "^env\.user.*",          "env.user='{}'".format(username))
    replace(fabfile, "^env\.key_filename.*",  "env.key_filename='{}'".format(os.path.join(keyDir,username)))
    replace(fabfile, "^env\.roledefs.*",      "env.roledefs={{'{}':['{}'],".format('stage',ip))

#Testing connection
#Adding key to remote machine
p = Popen("ssh -i {0}/{1} {1}@{2} -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no".format(keyDir,username,ip),shell=True,stdout=PIPE,stdin=PIPE)
print(p.communicate())

sed -i s/^env\.hosts.*/env.hosts=\['35.237.12.54']/ fabfile.py
sed -i s/^env\.user.*/env.user='tadhg'/ fabfile.py
sed -i s$^env\.key_filename.*$env\.key_filename='/home/jose-luis/.ssh/gotmKeys_feeagh/tadhg'$ fabfile.py
sed -i s/^env\.roledefs.*/env.roledefs={\'stage\':[\'35.237.12.54\'],/ fabfile.py
(b'Linux gotm-ssd 4.9.0-8-amd64 #1 SMP Debian 4.9.110-3+deb9u4 (2018-08-21) x86_64\n\nThe programs included with the Debian GNU/Linux system are free software;\nthe exact distribution terms for each program are described in the\nindividual files in /usr/share/doc/*/copyright.\n\nDebian GNU/Linux comes with ABSOLUTELY NO WARRANTY, to the extent\npermitted by applicable law.\n', None)


## Setting up GOTM and FABM in the remote machine

In [4]:
#ACPy .whl to be copied and installed on the remote machine
acpy_file = 'acpy-0.2-py2-none-any.whl'
schema_zip = 'GOTM_Schema.zip'
#callPopen('fab testConnection')
if not instanceExists:
    callPopen('fab getUtilities')
    callPopen('fab installACPy:{}'.format(acpy_file))
    callPopen('fab downloadModels') #note that the location of the key file is hardcoded in the getFABM function
    callPopen('fab upload:{},{}'.format(schema_zip,'~/code/schemas'))
    callPopen('fab compileModels')
    callPopen('fab getPROGNOS')
    callPopen('fab loadForcings')
callPopen('fab testRun:/home/{}/PROGNOS/langtjern/langtjern.xml'.format(username))

fab getUtilities
[35.237.12.54] Executing task 'getUtilities'
[35.237.12.54] Executing task 'update'
[35.237.12.54] Executing task 'updateMachine'
[35.237.12.54] run: sudo apt-get update
[35.237.12.54] out: 
[35.237.12.54] out: 0% [Working]
[35.237.12.54] out:             
[35.237.12.54] out: Get:1 http://security.debian.org stretch/updates InRelease [94.3 kB]
[35.237.12.54] out: 
[35.237.12.54] out: 0% [Waiting for headers] [1 InRelease 13.5 kB/94.3 kB 14%] [Waiting for headers
[35.237.12.54] out:                                                                                
[35.237.12.54] out: Ign:2 http://deb.debian.org/debian stretch InRelease
[35.237.12.54] out: 
[35.237.12.54] out: 0% [Waiting for headers] [1 InRelease 13.5 kB/94.3 kB 14%] [Waiting for headers
[35.237.12.54] out:                                                                                
[35.237.12.54] out: Get:3 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
[35.237.12.54] out: 
[35.237.12

[35.237.12.54] out: 0% [7 InRelease gpgv 3,843 B] [Waiting for headers]
[35.237.12.54] out:                                                    
[35.237.12.54] out: 0% [Waiting for headers]
[35.237.12.54] out: 0% [8 InRelease gpgv 3,876 B] [Waiting for headers] [Waiting for headers]
[35.237.12.54] out:                                                                          
[35.237.12.54] out: Get:21 http://packages.cloud.google.com/apt google-compute-engine-stretch-stable/main amd64 Packages [1,122 B]
[35.237.12.54] out: 
[35.237.12.54] out: 0% [8 InRelease gpgv 3,876 B] [Waiting for headers] [21 Packages 1,122 B/1,122 
[35.237.12.54] out:                                                                                
[35.237.12.54] out: 0% [8 InRelease gpgv 3,876 B] [Waiting for headers]
[35.237.12.54] out: 0% [21 Packages store 0 B] [8 InRelease gpgv 3,876 B] [Waiting for headers]
[35.237.12.54] out:                                                                            
[35.237

[35.237.12.54] out: 89% [68 Packages 222 B/222 B 100%]
[35.237.12.54] out:                                   
[35.237.12.54] out: 89% [Waiting for headers]
[35.237.12.54] out:                          
[35.237.12.54] out: Get:69 http://deb.debian.org/debian stretch-backports/main amd64 Packages 2018-09-22-0210.06.pdiff [485 B]
[35.237.12.54] out: 
[35.237.12.54] out: 89% [69 Packages 485 B/485 B 100%]
[35.237.12.54] out:                                   
[35.237.12.54] out: 90% [Waiting for headers]
[35.237.12.54] out:                          
[35.237.12.54] out: Get:70 http://deb.debian.org/debian stretch-backports/main amd64 Packages 2018-09-22-2008.27.pdiff [226 B]
[35.237.12.54] out: 
[35.237.12.54] out: 90% [Waiting for headers]
[35.237.12.54] out:                          
[35.237.12.54] out: Get:71 http://deb.debian.org/debian stretch-backports/main amd64 Packages 2018-09-23-0208.39.pdiff [588 B]
[35.237.12.54] out: 
[35.237.12.54] out: 90% [Waiting for headers]
[35.237.12.54]

[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 52%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 52%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 88%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 88%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 91%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 91%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 93%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 93%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 93%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 93%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 93%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 93%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 96%
[35.237.12.54] out: 
[35.237.12.54] out: Reading package lists... 96%
[35.237.12.54] out: 

[35.237.12.54] out: 6% [5 libperl5.24 449 kB/3,522 kB 13%]
[35.237.12.54] out:                                       
[35.237.12.54] out: 9% [Waiting for headers]
[35.237.12.54] out:                         
[35.237.12.54] out: Get:6 http://deb.debian.org/debian stretch/main amd64 perl amd64 5.24.1-3+deb9u4 [218 kB]
[35.237.12.54] out: 
[35.237.12.54] out: 9% [6 perl 31.7 kB/218 kB 15%]
[35.237.12.54] out:                               
[35.237.12.54] out: 9% [Waiting for headers]
[35.237.12.54] out:                         
[35.237.12.54] out: Get:7 http://deb.debian.org/debian stretch/main amd64 cmake-data all 3.7.2-1 [1,216 kB]
[35.237.12.54] out: 
[35.237.12.54] out: 9% [7 cmake-data 13.9 kB/1,216 kB 1%]
[35.237.12.54] out:                                      
[35.237.12.54] out: Get:8 http://deb.debian.org/debian stretch/main amd64 liblzo2-2 amd64 2.08-1.2+b2 [55.0 kB]
[35.237.12.54] out: 
[35.237.12.54] out: 11% [8 liblzo2-2 19.0 kB/55.0 kB 35%]
[35.237.12.54] out:              

...

[35.237.12.54] out: Receiving objects:  21% (489/2281), 59.07 MiB | 11.50 MiB/s   
[35.237.12.54] out: Receiving objects:  22% (502/2281), 68.31 MiB | 13.06 MiB/s   
[35.237.12.54] out: Receiving objects:  23% (525/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  24% (548/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  25% (571/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  26% (594/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  27% (616/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  28% (639/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  29% (662/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  30% (685/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  30% (688/2281), 77.04 MiB | 14.41 MiB/s   
[35.237.12.54] out: Receiving objects:  31% (708/2281), 77.04 MiB | 14.41 MiB/s   
[35.

[35.237.12.54] out: Resolving deltas:  30% (365/1208)   
[35.237.12.54] out: Resolving deltas:  31% (376/1208)   
[35.237.12.54] out: Resolving deltas:  32% (389/1208)   
[35.237.12.54] out: Resolving deltas:  33% (399/1208)   
[35.237.12.54] out: Resolving deltas:  34% (411/1208)   
[35.237.12.54] out: Resolving deltas:  36% (435/1208)   
[35.237.12.54] out: Resolving deltas:  37% (449/1208)   
[35.237.12.54] out: Resolving deltas:  38% (460/1208)   
[35.237.12.54] out: Resolving deltas:  39% (473/1208)   
[35.237.12.54] out: Resolving deltas:  40% (484/1208)   
[35.237.12.54] out: Resolving deltas:  41% (497/1208)   
[35.237.12.54] out: Resolving deltas:  42% (508/1208)   
[35.237.12.54] out: Resolving deltas:  43% (520/1208)   
[35.237.12.54] out: Resolving deltas:  49% (592/1208)   
[35.237.12.54] out: Resolving deltas:  50% (604/1208)   
[35.237.12.54] out: Resolving deltas:  51% (621/1208)   
[35.237.12.54] out: Resolving deltas:  52% (629/1208)   
[35.237.12.54] out: Resolving d

[35.237.12.54] Executing task 'testRun'
[35.237.12.54] run: cd "$(dirname /home/tadhg/PROGNOS/langtjern/langtjern.xml)" && editscenario --schemadir "$GOTMDIR"/schemas -e nml . langtjern.xml
[35.237.12.54] out: Operating on values file with version gotm-5.1.
[35.237.12.54] out: Scenario validated successfully.
[35.237.12.54] out: Exporting values to namelist file(s) ....
[35.237.12.54] out: 

[35.237.12.54] run: cd "$(dirname "/home/tadhg/PROGNOS/langtjern/langtjern.xml")" && gotm
[35.237.12.54] out:  ------------------------------------------------------------------------
[35.237.12.54] out:  GOTM started on 2018/09/26 at 14:21:10
[35.237.12.54] out:  ------------------------------------------------------------------------
[35.237.12.54] out:     init_gotm
[35.237.12.54] out:  ------------------------------------------------------------------------
[35.237.12.54] out:         reading model setup namelists..
[35.237.12.54] out:         done.
[35.237.12.54] out:         GOTM Simulation
[

[35.237.12.54] out:             cyanobacteria_po_sms  mmol P/m3/s  cyanobacteria phosphate sources-sinks
[35.237.12.54] out:             cyanobacteria_dd_sms  mmol N/m3/s  cyanobacteria detritus sources-sinks
[35.237.12.54] out:             cyanobacteria_mole_concentration_of_dissolved_inorganic_carbon_sms  mmol m-3/s  cyanobacteria mole_concentration_of_dissolved_inorganic_carbon sources-sinks
[35.237.12.54] out:             cyanobacteria_chla  mg chl a/m3  cyanobacteria chlorophyll concentration
[35.237.12.54] out:             cyanobacteria_GPP  mmol/m3/d  cyanobacteria gross primary production
[35.237.12.54] out:             cyanobacteria_NPP  mmol/m3/d  cyanobacteria net primary production
[35.237.12.54] out:             zooplankton_c_sms  mmol N/m3/s  zooplankton concentration sources-sinks
[35.237.12.54] out:             zooplankton_c_w  m/s  zooplankton concentration vertical movement
[35.237.12.54] out:             zooplankton_prey1_sms  mmol N/m3/s  zooplankton prey 1 sources-

[35.237.12.54] out:           - v
[35.237.12.54] out:           - xP
[35.237.12.54] out:           - h
[35.237.12.54] out:           - tke
[35.237.12.54] out:           - eps
[35.237.12.54] out:           - num
[35.237.12.54] out:           - nuh
[35.237.12.54] out:           - nus
[35.237.12.54] out:           - dom_DOMa
[35.237.12.54] out:           - dom_DOMb
[35.237.12.54] out:           - diatoms_c
[35.237.12.54] out:           - flagellates_c
[35.237.12.54] out:           - cyanobacteria_c
[35.237.12.54] out:           - zooplankton_c
[35.237.12.54] out:           - selma_dd
[35.237.12.54] out:           - selma_aa
[35.237.12.54] out:           - selma_nn
[35.237.12.54] out:           - selma_po
[35.237.12.54] out:           - selma_o2
[35.237.12.54] out:           - selma_pw
[35.237.12.54] out:           - selma_fl
[35.237.12.54] out:           - selma_pb
[35.237.12.54] out:           - u_obs
[35.237.12.54] out:           - v_obs
[35.237.12.54] out:           - fric
[35.237.12.5

## Selecting parameters to calibrate

### Selecting parameters to calibrate

In [5]:
parameterDict = {'shf_factor' : 
            {'file' : 'airsea.nml', 'namelist': 'airsea', 'minimum' : 0.5, 'maximum' : 1.5, 'cal' : True} }
parameterDict['swr_factor'] =  {'file' : 'airsea.nml', 'namelist': 'airsea', 'minimum' : 0.5, 'maximum' : 1.5, 'cal' : True} 
parameterDict['wind_factor'] = {'file' : 'airsea.nml', 'namelist': 'airsea', 'minimum' : 0.0, 'maximum' : 3.0, 'cal' : True} 
parameterDict['k_min'] = {'file' : 'gotmturb.nml', 'namelist': 'turb_param', 'minimum' : 1e-9, 'maximum' : 1e-5, 'logscale' : 'True', 'cal' : True} 
parameterDict['A'] = {'file' : 'obs.nml', 'namelist': 'extinct', 'minimum' : 0.0, 'maximum' : 1.0, 'cal' : True} 
parameterDict['g1'] = {'file' : 'obs.nml', 'namelist': 'extinct', 'minimum' : 0.1, 'maximum' : 3.0, 'cal' : True} 
parameterDict['g2'] = {'file' : 'obs.nml', 'namelist': 'extinct', 'minimum' : 1.0, 'maximum' : 30.0, 'cal' : True}
parameterDict['instances/dom/parameters/kOM2'] = {'file' : 'fabm.yaml', 'minimum' : 0., 'maximum' : 1.1, 'cal' : True} 
parameterDict['instances/dom/parameters/kOM1'] = {'file' : 'fabm.yaml', 'minimum' : 0., 'maximum' : 300., 'cal' : True} 
parameterDict['instances/dom/parameters/Km_O2'] = {'file' : 'fabm.yaml', 'minimum' : 0.1, 'maximum' : 20., 'cal' : True} 
parameterDict['instances/dom/parameters/Km_NO3'] = {'file' : 'fabm.yaml', 'minimum' : 1., 'maximum' : 10., 'cal' : True} 
parameterDict['instances/dom/parameters/Kin_O2'] = {'file' : 'fabm.yaml', 'minimum' : 1., 'maximum' : 10., 'cal' : True}
#parameterDict['instances/dom/parameters/qy_DOM'] = {'file' : 'fabm.yaml', 'minimum' : 1e-5, 'maximum' : 1e-2, 'cal' : True, 'logscale' : True}
#parameterDict['instances/dom/parameters/theta'] = {'file' : 'fabm.yaml', 'minimum' : 0.5, 'maximum' : 1.5, 'cal' : True}
#parameterDict['instances/dom/parameters/k_floc'] = {'file' : 'fabm.yaml', 'minimum' : 1E-14, 'maximum' : 1.5, 'cal' : True} 
print(parameterDict)

{'shf_factor': {'file': 'airsea.nml', 'namelist': 'airsea', 'minimum': 0.5, 'maximum': 1.5, 'cal': True}, 'swr_factor': {'file': 'airsea.nml', 'namelist': 'airsea', 'minimum': 0.5, 'maximum': 1.5, 'cal': True}, 'wind_factor': {'file': 'airsea.nml', 'namelist': 'airsea', 'minimum': 0.0, 'maximum': 3.0, 'cal': True}, 'k_min': {'file': 'gotmturb.nml', 'namelist': 'turb_param', 'minimum': 1e-09, 'maximum': 1e-05, 'logscale': 'True', 'cal': True}, 'A': {'file': 'obs.nml', 'namelist': 'extinct', 'minimum': 0.0, 'maximum': 1.0, 'cal': True}, 'g1': {'file': 'obs.nml', 'namelist': 'extinct', 'minimum': 0.1, 'maximum': 3.0, 'cal': True}, 'g2': {'file': 'obs.nml', 'namelist': 'extinct', 'minimum': 1.0, 'maximum': 30.0, 'cal': True}, 'instances/dom/parameters/kOM2': {'file': 'fabm.yaml', 'minimum': 0.0, 'maximum': 1.1, 'cal': True}, 'instances/dom/parameters/kOM1': {'file': 'fabm.yaml', 'minimum': 0.0, 'maximum': 300.0, 'cal': True}, 'instances/dom/parameters/Km_O2': {'file': 'fabm.yaml', 'minimum

In [6]:
#Function to generate XML config file for ACPy

def generate_XML_config(filename,dummy):
    document = ET.Element('config')
    transports = ET.SubElement(document, 'transports')
    database = ET.SubElement(transports,'transport', type='sqlite',path='langtjern.db')
    executable = ET.SubElement(document, 'executable', path='/usr/bin/gotm')
    setup_path = ET.SubElement(document, 'setup', path='../')
    parameters = ET.SubElement(document, 'parameters')
    observations = ET.SubElement(document, 'observations')

    #Adding parameters to calibrate
    for key, value in parameterDict.items():
        print(key,value['cal'])
        if value['cal']:
            temp = value
            #temp.rename(columns={"min": "minimum", "max": "maximum"})
            temp.pop('cal',None)
            temp['minimum'] = str(temp['minimum'])
            temp['maximum'] = str(temp['maximum'])
            print(temp)
            ET.SubElement(parameters, 'parameter', attrib=temp, variable=key)
            

    #Adding observations to calibrate against
    ET.SubElement(observations, 'variable',
                  source="./temp.obs",
                  modelpath="output.nc",
                  modelvariable="temp",
                  spinupyears="0"
                 )

   # ET.SubElement(observations, 'variable',
   #               source="./cdom_out.obs",
   #               modelpath="output.nc",
   #               modelvariable="dom_DOMb",
   #               spinupyears="0"
   #              )


    et = ET.ElementTree(document)

    f = BytesIO()
    et.write(f, encoding='utf-8', xml_declaration=True) 
    bla =f.getvalue()

    with open(filename,'wb') as out: ## Open temporary file as bytes
        out.write(bla) 
        
    print(bla)    

In [7]:
xml_filename = 'config_acpy.xml'
def on_generate_xml_clicked(filename,dummy):
    for i in items_checkbox : 
        if i.value :
            parameterDict[i.description]['cal'] = True
        else :
            parameterDict[i.description]['cal'] = False
    generate_XML_config(filename,parameterDict)   
    #print("Button clicked.")

items_checkbox = [  widgets.Checkbox(
                    value=True,
                    description=key,
                    disabled=False,
                    layout=widgets.Layout(width='350px', grid_area = ('check_' + key).replace("/","_"), justify = 'end')
                )               
                for key, value in parameterDict.items()
                ]

items_range = [ widgets.FloatRangeSlider(
                    value=[float(value['minimum']), float(value['maximum'])],
                    min=float(value['minimum']),
                    max=float(value['maximum']),
                    step=(float(value['maximum']) - float(value['minimum'])) / 100.,
                    description='',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    layout=widgets.Layout(width='auto', grid_area = ("range_" + key).replace("/","_") )
                )    
                for key, value in parameterDict.items()
                ]


items_range_min = [ widgets.FloatRangeSlider(
                    value=[float(value['minimum']), float(value['maximum'])],
                    min=float(value['minimum']),
                    max=float(value['maximum']),
                    step=(float(value['maximum']) - float(value['minimum'])) / 100.,
                    description='',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='.2f',
                    layout=widgets.Layout(width='auto', grid_area = ("range_" + key).replace("/","_") )
                )    
                for key, value in parameterDict.items()
                ]




header  = widgets.Button(description='Select parameters to calibrate',
                 layout=widgets.Layout(width='auto', grid_area='header'),
                 style=widgets.ButtonStyle(button_color='lightblue'))


footer  = widgets.Button(description='Generate XML',
                 layout=widgets.Layout(width='auto', grid_area='footer'),
                 style=widgets.ButtonStyle(button_color='olive'))

footer.on_click(partial(on_generate_xml_clicked,xml_filename))

boxChildren = [header,] + items_checkbox + items_range + [footer,]

template_areas = '"header header"'
for key,val in parameterDict.items():
    template_areas += '"check_{0} range_{0}"'.format(key).replace("/","_")

template_areas += '"footer footer"'
# print(template_areas)

widgets.GridBox(children=boxChildren,
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto ' * (len(parameterDict)),
            grid_template_columns='40% 60%',
            grid_template_areas=template_areas
            )
            )
          

GridBox(children=(Button(description='Select parameters to calibrate', layout=Layout(grid_area='header', width…

In [8]:
callPopen('fab runACPy:{}'.format(xml_filename))

fab runACPy:config_acpy.xml
[35.237.12.54] Executing task 'runACPy'
[35.237.12.54] put: config_acpy.xml -> ./PROGNOS/langtjern/acpy/config_acpy.xml

Done.
Disconnecting from 35.237.12.54... done.
